## 1.0 Prerequisites on libraries

In [ ]:
## Library import
import os

## Libraries for basic data-processing & visualization
import numpy as np
import pandas as pd
!pip install -U seaborn
import seaborn as sns

## Libraries for decision-tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from IPython.display import Image
import graphviz
from sklearn.externals.six import StringIO
os.environ["PATH"] += os.pathsep + 'C:\Program Files (x86)\Graphviz2.38/bin/'
try:
    import pydotplus
except:
    !pip install pydotplus
    import pydotplus
    
## Libraries for logistic-regression
from sklearn.linear_model import LogisticRegression

## Library for evaluation on classification
from sklearn.metrics import confusion_matrix, f1_score

## Libraries for Keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

## Random seed for reproductivity, especially for Keras
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
WKDIR = '../data/'
os.chdir(WKDIR)

## 2.0 Load data & its definition

In this chapter, we load raw data from csv-format data. Original data is [this site](https://www.kaggle.com/johndddddd/customer-satisfaction).

Also, we review the basic functions on pandas & numpy in comparing SQL syntax. Please note in advance that they don't cover all functions.

In [ ]:
## Data import
SATISFACTION_FILE = 'satisfaction.csv'
df = pd.read_csv(SATISFACTION_FILE)

In [ ]:
## Size of DataFrame (row numbers, column numbers)
df.shape

In [ ]:
## Confirm data columns
df.columns

### Definition of each column


|Column|Description|Data examples|
|-----|:-----:|:-----|
|satisfaction_v2|Airline satisfaction level|Satisfaction, neutral or dissatisfaction|
|Gender|Gender of the passengers |Female, Male|
|Customer Type|The customer type|Loyal customer, disloyal customer|
|Age|The actual age of the passengers||
|Type of Travel|Purpose of the flight of the passengers|Personal Travel, Business Travel|
|Class|Travel class in the plane of the passengers|Business, Eco, Eco Plus|
|Flight Distance|The flight distance of this journey||
|Seat comfort|Satisfaction level of Seat comfort||
|Departure/Arrival time convenient|Satisfaction level of Departure/Arrival time convenient||
|Food and drink|Satisfaction level of Food and drink||
|Gate location|Satisfaction level of Gate location||
|Inflight wifi service|Satisfaction level of the inflight wifi service|0:Not Applicable;1-5|
|Inflight entertainment|Satisfaction level of inflight entertainment||
|Online support|??||
|Ease of Online booking|Satisfaction level of online booking||
|On-board service|Satisfaction level of On-board service||
|Leg room service|Satisfaction level of Leg room service||
|Baggage handling|Satisfaction level of baggage handling||
|Checkin service|Satisfaction level of Check-in service||
|Cleanliness|Satisfaction level of Cleanliness||
|Online boarding|Satisfaction level of online boarding||
|Departure Delay in Minutes|Minutes delayed when departure||
|Arrival Delay in Minutes|Minutes delayed when Arrival|



Ref. https://www.kaggle.com/johndddddd/customer-satisfaction/home

### 2.0.1 Projection

If SQL, ...
```SQL
SELECT
    TOP(10) [Flight Distance]
FROM
    TABLE_SATISFACTION
```

In [ ]:
df['Flight Distance'].head(10)

### 2.0.2 Projection with plural columns
If SQL, ...
```SQL
SELECT
    TOP(5) [Flight Distance], [Type of Travel]
FROM
    TABLE_SATISFACTION
```

In [ ]:
df[['Flight Distance', 'Type of Travel']].head()

### 2.0.3 Select distinct

If SQL, ...

```SQL
SELECT
    DISTINCT [Flight Distance]
FROM
    TABLE_SATISFACTION
```

In [ ]:
df['Flight Distance'].unique()

### 2.0.4 filter

If SQL, ...

```SQL
SELECT
    TOP(10) *
FROM
    TABLE_SATISFACTION
WHERE
    Gender = 'Female'
```

In [ ]:
df.query("Gender == 'Female'").head(10)

### 2.0.5 order by

If SQL, ...
```SQL
SELECT
    *
FROM
    TABLE_SATISFACTION
ORDER BY
    [Flight Distance]
```

In [ ]:
df.sort_values('Flight Distance').head(10)

### 2.0.6 group by & aggregate function

If SQL, ...
```SQL
SELECT
    AVG([Flight Distance])
    ,[Type of Travel]
FROM
    TABLE_SATISFACTION
GROUP BY
    [Type of Travel]
```

In [ ]:
df[['Flight Distance', 'Type of Travel']].groupby('Type of Travel').mean()

### 2.0.7 Case statement

If SQL, ...
```SQL
SELECT
    CASE [satisfaction_v2]
        WHEN 'satisfied' THEN 1
        ELSE 0
    END AS [target]
FROM
    TABLE_SATISFACTION
```

In [ ]:
df['target'] = df['satisfaction_v2'].apply(lambda x: 1 if x == 'satisfied' else 0)

### 2.0.8 group by & count

If SQL, ...
```SQL
SELECT
    COUNT(1)
    ,[Type of Travel]
FROM
    TABLE_SATISFACTION
GROUP BY
    [Type of Travel]
```

In [ ]:
df['Type of Travel'].value_counts()

### 2.0.9 Populate dummy variables

If SQL, ...
```SQL
SELECT
    CASE [Type of Travel]
        WHEN 'Business trave' THEN 1
        ELSE 0
    END AS [Business trave]
    ,CASE [Type of Travel]
        WHEN 'Personal Travel' THEN 1
        ELSE 0
    END AS [Personal Travel]
FROM
    TABLE_SATISFACTION
```

In [ ]:
pd.get_dummies(df['Type of Travel']).head()

### 2.0.10 pivot table

In [ ]:
pd.pivot_table(df, values=['Flight Distance', 'Seat comfort', 'Online boarding'], index='Type of Travel', aggfunc='mean')

## 2.1 Slice on dataframe

In [ ]:
## Extract first 10 rows
## Attention!! index of python starts from 0.
df[:10]

In [ ]:
## Extract 15th row to 20th row
df[15:20]

In [ ]:
## Use iloc, If we focus on specific columns
df.iloc[15:20, [4]]

In [ ]:
## Or, slice after projection by pandas
df['Age'][15:20]

In [ ]:
## The above 2 objects are different, though...
print(type(df.iloc[15:20, [4]]))
print(type(df['Age'][15:20]))

## 2.2 Basic summary

### 2.2.1 Basic statistics for each column

In [ ]:
df.describe().T

### 2.2.2 Check 'NA'(Not Applicable) value for each column

In [ ]:
df.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int).T

## 2.3 Viaualize & understanding data

### 2.3.1 Check distribution on single colum

In [ ]:
## Visualize Flight distance
sns.distplot(df['Flight Distance'], kde=True)

In [ ]:
## To-Do: Put another column name on argument of the dataframe 
col_single = 'Food and drink'
sns.distplot(df[col_single], kde=False)

### 2.3.2 Understand relation of several columns

In [ ]:
## Calculate correlational coefficient
np.corrcoef(df['Seat comfort'], df['Food and drink'])[1,0]

In [ ]:
## To-Do: ut other columns on argument of calculating
col1_corr = 'Inflight wifi service'
col2_corr = 'Inflight entertainment'

np.corrcoef(df[col1_corr], df[col2_corr])[1,0]

In [ ]:
## Correlation matrix for some variables

col_corr = ['Flight Distance'
            ,'Seat comfort'
            ,'Food and drink'
            ,'Inflight wifi service'
            ,'Inflight entertainment'
            ,'Online support'
            ,'Cleanliness'
            ,'Ease of Online booking'
            ,'Departure Delay in Minutes']

plt.figure(figsize=(16,10))
sns.heatmap(df[col_corr].corr(), annot=True)

### 2.3.3 Scatter plot on 2 variables

In [ ]:
## Try to depict scatter plot, ...
sns.scatterplot(df['Food and drink'], df['Leg room service']) ## variant is too small ..:(

In [ ]:
#sns.jointplot(df['Food and drink'], df['Leg room service'], kind="kde")

In [ ]:
## Calculate correlational coefficient
np.corrcoef(df['Food and drink'], df['Leg room service'])[1,0]

In [ ]:
def depict_corr_matrix(col1, col2):
    ## Calculate & print correlation coefficient
    corr = np.corrcoef(df[col1], df[col2])[1,0]
    print('Correlational coefficient is {}'.format(corr))
    ## Depict joint plot
    sns.jointplot(df[col1], df[col2], kind="kde")

In [ ]:
## In summary, ...
#depict_corr_matrix('Food and drink', 'Leg room service')

In [ ]:
## Try another combination
#depict_corr_matrix('Inflight wifi service', 'Inflight entertainment')

In [ ]:
## Try another combination
#depict_corr_matrix('Food and drink', 'Seat comfort')

## 3.0 Machine Learning

In this chapter, we consider statistical model, which predict passengers' satisfaction with other variables.

- Goal of this chapter
  - Select some given variables and confirm which variable impacts the satisfaction as a whole.
    - Decision Tree Classifier
    - Logistic Classifier

- Notes:
  - In order to move forward, we put `target` as target variable: 1: `satisfied`, 0: `neutral or dissatisfied`
  - In next chapter, we pursuit the predictivity with deep-learning technique and compare the accuracy with logistic classifier.

## 3.1 Decision Tree classifier

In [ ]:
col_ml= ['Flight Distance'
        ,'Seat comfort'
        ,'Food and drink'
        ,'Inflight wifi service'
        ,'Inflight entertainment'
        ,'Online support'
        ,'Ease of Online booking'
        ,'Departure Delay in Minutes']

In [ ]:
## Define decision tree model
clf = DecisionTreeClassifier(random_state=0, max_depth=4)

In [ ]:
## Split the whole data into train & test
Obs_for_train = 120000
X_train = df[col_ml][:Obs_for_train]
y_train = df['target'][:Obs_for_train]
X_test = df[col_ml][Obs_for_train:]
y_test = df['target'][Obs_for_train:]

In [ ]:
## Optimize parameters of the model
dt = clf.fit(X_train, y_train)

In [ ]:
## Calculate accuracy
accuracy_dt = dt.score(X_test, y_test)
print(accuracy_dt)

In [ ]:
## Check feature importance for eadh variable
dt.feature_importances_

In [ ]:
sns.set()
plt.figure(figsize=(15,10))
sns.barplot(col_ml, dt.feature_importances_)

In [ ]:
## F1 score
f1_score(y_test, dt.predict(X_test))

In [ ]:
## Depict tree
dot_data = StringIO()
tree.export_graphviz(dt 
                     ,out_file=dot_data
                     ,feature_names=col_ml
                     ,filled=True
                     ,rounded=True
                     ,special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

## Logistic regression

In [ ]:
## Define logistic regression
clf_log = LogisticRegression(random_state=0, solver='lbfgs', max_iter=100)

In [ ]:
## Optimize parameters of the model
clf_log.fit(X_train, y_train)

In [ ]:
## Calculate accuracies
accuracy_lr = clf_log.score(X_test, y_test)
print(accuracy_lr)

In [ ]:
## Each partial regression coefficient
clf_log.coef_

In [ ]:
## Compare partial regression coefficients
sns.set()
plt.figure(figsize=(15,10))
sns.barplot(col_ml, clf_log.coef_[0])
plt.title('Partial regression coefficient on logistic regression')

In [ ]:
## Confusiton matrix
confusion_matrix(y_test, clf_log.predict(X_test))

In [ ]:
## F1 score
f1_score(y_test, clf_log.predict(X_test))

## 3.3 Deep Learning

- In this chapter, we'd like to pursuit predictive accuracy with deep-learning technique, especially MLP(=Multi-Layer Perceptron). Please treat this content as introduction for deep-learning.

### 3.3.1 Equivalent to logistic regression

In [ ]:
model1 = Sequential()
model1.add(Dense(1, input_dim=X_train.shape[1], activation='sigmoid'))

model1.compile(loss='binary_crossentropy'
             ,optimizer='adam'
             ,metrics=['accuracy'])

In [ ]:
history1 = model1.fit(X_train, y_train, epochs=100, batch_size=1001, verbose=1, shuffle=False)

In [ ]:
plt.plot(history1.history['loss'], label='train')
plt.legend()
plt.xlabel('epoch number')
plt.ylabel('loss')
plt.show()

In [ ]:
loss1, accuracy_dl1 = model1.evaluate(X_test, y_test)
print('loss: {}, accuracy: {}'.format(loss1, accuracy_dl1))

In [ ]:
model1.summary()

### 3.3.2 MLP -- Adding layers

In [ ]:
model2 = Sequential()
model2.add(Dense(5, input_dim=X_train.shape[1], activation='relu'))
model2.add(Dense(5, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy'
             ,optimizer='adam'
             ,metrics=['accuracy'])

In [ ]:
history2 = model2.fit(X_train, y_train, epochs=100, batch_size=1000, verbose=1, shuffle=False)

In [ ]:
plt.plot(history2.history['loss'], label='train')
plt.legend()
plt.xlabel('epoch number')
plt.ylabel('loss')
plt.show()

In [ ]:
loss2, accuracy_dl2 = model2.evaluate(X_test, y_test)
print('loss: {}, accuracy: {}'.format(loss2, accuracy_dl2))

In [ ]:
model2.summary()

## 3.4 Compare accuracies for all models

In [ ]:
model_name = ['Decision Tree', 'Logistic regression', 'equivalent to Log-reg', 'MLP']
sns.set()
plt.figure(figsize=(15,10))
sns.barplot(model_name, [accuracy_dt, accuracy_lr, accuracy_dl1, accuracy_dl2])